# Preamble

In [ ]:
strain='rdv2'
process = 'wholepop'
programs = ['sv-fas','sv-ras']

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy.interpolate import interp1d

%matplotlib inline
plt.viridis()

data_path = "../cph8-ompr_data"

import sys
sys.path.extend(["../../lsc"])
import lsc
import plotting_resources as pr

spectra_path = '../../led_data/led_reference_data/'
sv_led_names = ['R12','R120','R2007','R27','R3150','R3310','R39','R90']
sv_dfs = []
for ln in sv_led_names:
    fn = os.path.join(spectra_path, '{}.xls'.format(ln))
    sv_dfs.append(pd.read_excel(fn))

white = 159.4

dfs = {}
for pt in programs:
    if pt in ['sv-fas', 'sv-ras']:
        pt_true = pt
        pt = 'sv'
    prog_path = os.path.join(data_path, pt)
    xl_path = os.path.join(prog_path, '{}_{}_{}_complete.xlsx'.format(strain,process,pt))
    pt = pt_true
    
    df = pd.read_excel(xl_path)
    df['FL1 Mean'] = df['FL1 Mean'] - white
    
    if pt == 'dv-mono':
        dfs['dv-mono-g'] = df.loc[df['Bot LED ID'] == '520-2-KB']
        dfs['dv-mono-uv'] = df.loc[df['Bot LED ID'] == '380-SB']
    else:
        dfs[pt] = df
        
    if pt in ['sv-fas', 'sv-ras']:
        if pt == 'sv-fas':
            actint = 0
        elif pt == 'sv-ras':
            actint = 1.25
        
        newdf = dfs[pt][dfs[pt]['Activating intensity'] == actint]
        dfs[pt] = newdf
        spec_int_triples = [[float(y) for y in x.split(',')] for x in dfs[pt]['Spectral intensities']]
        well_nums = dfs[pt]['Pre-rand well'].values
        dfs[pt]['Spectral intensity'] = [spec_int_triples[i][np.mod(well_nums[i],3)] for i in range(len(well_nums))]
        
# Cleaning data
eliminated_samples = [
]

for es in eliminated_samples:
    df = dfs[es[0]]
    idx = df.loc[df['Sample ID'] == es[1]].index.tolist()[0]
    df.at[idx, 'FL1 Mean'] = np.nan
        
prog_signals_path = "../../gene_expression_programming_signals"
ref_path = os.path.join(prog_signals_path, 'programming_reference_signals.xlsx')
ref_df = pd.read_excel(ref_path)

genprog_path = os.path.join(prog_signals_path, 'generated_light_programs.xls')
genprog_df = pd.read_excel(genprog_path)
        
## Set the default plot output to be inline in this notebook
%matplotlib inline

## Initialize plotting style
pr.init_style()

## Set plotting output directory

out_folder = './plots/'

## Set default axes ranges, ticks, and labels for common axes
# Time
t_range = [0,360]
t_ticks = [0,120,240,360]
t_label = "Time (min)"

# GFP fluorescence
gfp_range = [0,5.5e3]
gfp_ticks = [0,1e3,2e3,3e3,4e3]
gfp_range_log = [3e1, 2e4]
gfp_ticks_log = [1e2, 1e3, 1e4]
gfp_label = "GFP fluorescence (MEFL)"

# Light intensity
int_range_lin = [0, 20]
int_range_log = [20 / 2**11, 20]
int_ticks_lin = [0, 10, 20]
int_ticks_log = [1e-2,1e-1,1e0,1e1]
int_label = "Light intensity (\si{\micro\mole.m^{-2}.s^{-1}})"

## Set up color cycles
red_cycle = pr.red_cycle
led_cycle = pr.led_colors
sv_cycle = ['#c7cd31','#ed474c','#7486f2','#c11d36','#a8a673','#d38f89','#dd29e3','#16c845']

## Copy predicted photoconversion rates from PCS spline construction

In [ ]:
# Values with 526nm LED included during PCS construction
k1s = [0.015015623053265937, 0.025255878161038664, 0.0053281202299974842, 0.017852337351979105, 0.01425680446168328, 0.018719738689186621, 0.020636223218596636, 0.0037371229578212215]
k2s = [0.001181906721348529, 0.002154960863399176, 0.0039944202807868558, 0.003624363217405131, 0.0013247914347169743, 0.0013385430489715141, 0.002711732346948127, 0.0013104540423658616]

sv_info = zip(sv_led_names, k1s, k2s)

## Simulate response to spectral validation LEDs

In [ ]:
datasets = ['sv-fas','sv-ras']
reload(lsc)
m = lsc.Model.from_path('../cph8-ompr_model_parameters/rdv2_m01/all_data_onetau_with_kholds_and_adjusted_initial.params', hill_type='rep')
m.params['b'].value = m.params['b'].value - white

for sv_led in sv_info:
    name, k1, k2 = sv_led
    m.ledids.append(name.lower())
    idx = m.ledids.index(name.lower())
    m.params.add('k1_{}'.format(idx), value=k1)
    m.params.add('k2_{}'.format(idx), value=k2)

sims = {}
for dset in datasets:
    df = dfs[dset].sort_values(['Spectral intensity','Top LED ID'])
    X = np.reshape(df['Top LED ID'], (12,8))
    Y = np.reshape(df['Spectral intensity'], (12,8))[:,0]
    Y = np.logspace(np.log10(Y[0]), np.log10(Y[-1]), 100)
    A = np.reshape(df['Activating intensity'], (12,8))
    Z = np.reshape(df['FL1 Mean'], (12,8))

    led_names = [x.split('White-1-SB_')[1] for x in X[0]]
    spec_ints = Y
    act_ints = [A[0,0]]*len(spec_ints)
    progs = [[0 for y in range(len(spec_ints))] for x in range(len(led_names))]
    sim = [[0 for y in range(len(spec_ints))] for x in range(len(led_names))]
    for i,x in enumerate(led_names):
        for j,y in enumerate(spec_ints):
            l1 = lsc.LEDProgram(led_names[i].lower(), spec_ints[j])
            l2 = lsc.LEDProgram('660-ls', act_ints[j])
            
            progs[i][j] = lsc.WellProgram.fromlist([l1, l2])
            sim[i][j] = m.simulate_well(progs[i][j])
    
    sims[dset] = np.array(sim)

## Plot response to spectral validation LEDs

In [ ]:
scales = ['log']

plt.rcParams['savefig.dpi'] = 600
plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['axes.color_cycle'] = led_cycle
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

plt.rcParams['font.size'] = 12
plt.rcParams['xtick.labelsize'] = plt.rcParams['font.size']
plt.rcParams['ytick.labelsize'] = plt.rcParams['font.size']

fig = plt.figure()

ax = fig.add_subplot(111)
ax.set_xlabel(int_label)
ax.set_ylabel(gfp_label)
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['right'].set_color('none')
ax.tick_params(labelcolor='w', top='off', bottom='off', left='off', right='off')

df1 = dfs['sv-fas'].sort_values(['Spectral intensity','Top LED ID'])
df2 = dfs['sv-ras'].sort_values(['Spectral intensity','Top LED ID'])

for scale in scales:
    for i in range(8):
        ax = fig.add_subplot(2,4,i+1)
    #     Xd = aas['spec_i_act'][:,0]
    #     Yd1 = aas['gm'][:,i]
    #     Yd2 = das['gm'][:,i]

    #     Xm = np.logspace(-3, 2, 100)
    #     Ym1 = [steadystate(results_aas_das.params,inten,        0, led1=i, led2=7,pcs1=pcs1,pcs2=pcs2) for inten in Xm]
    #     Ym2 = [steadystate(results_aas_das.params,inten,das_inten, led1=i, led2=7,pcs1=pcs1,pcs2=pcs2) for inten in Xm]


    #             X = np.reshape(df['Top Centroid (nm)'], (12,24))[:,:23]
    #             Y = np.reshape(df['Spectral intensity'], (12,24))[:,:23]

    #             if model in ['raw']:
    #                 Z = np.reshape(df['FL1 Mean'], (12,24))[:,:23]
    #                 Z[-3:,0] = np.nan
    #             if model in ['model']:
    #                 Z = sims[dset]

        imax = 13
        
        Xd1 = np.reshape(df1['Spectral intensity'], (12,8))[:imax,i]
        Xd2 = np.reshape(df2['Spectral intensity'], (12,8))[:imax,i]
        Yd1 = np.reshape(df1['FL1 Mean'], (12,8))[:imax,i]
        Yd2 = np.reshape(df2['FL1 Mean'], (12,8))[:imax,i]
        
        Xm = np.reshape(df1['Spectral intensity'], (12,8))[:,i]
        Xm = np.logspace(np.log10(Xm[0]), np.log10(Xm[-1]), 100)
        Ym1 = sims['sv-fas']
        Ym2 = sims['sv-ras']
        
        if scale in ['lin']:
            ax.semilogx(Xd1, Yd1, '.', color = pr.led_colors[i])
            ax.semilogx(Xd2, Yd2, '.', color = pr.led_colors[i], marker='s', ms=6, markeredgecolor='none')
            ax.semilogx(Xm, Ym1[i], '-', color = pr.led_colors[i])
            ax.semilogx(Xm, Ym2[i], '-', color = pr.led_colors[i])
            ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            ax.set_ylim(gfp_range)
            ax.set_yticks(gfp_ticks)
        
        if scale in ['log']:
            ax.loglog(Xd1, Yd1, '.', color = sv_cycle[i])
            ax.loglog(Xd2, Yd2, '.', color = sv_cycle[i], marker='s', ms=6, markeredgecolor='none')
            ax.loglog(Xm, Ym1[i], '-', color = sv_cycle[i])
            ax.loglog(Xm, Ym2[i], '-', color = sv_cycle[i])
            ax.set_ylim(plt.ylim([3e1,1e4])) 
            
        
        ax.set_xlim(int_range_log)
        ax.set_xticks(int_ticks_log)

        ax = plt.gca()
        ax.minorticks_on()
        ax.tick_params(axis='x',which='both',top='off')
        ax.tick_params(axis='y',which='both',right='off')

        ax = plt.gca()
        ax.text(0.5,0.95,\
                r'{}'.format(led_names[i]),\
                horizontalalignment='center',\
                verticalalignment='top',\
                fontsize = plt.rcParams['font.size'],
                transform=ax.transAxes)

    plt.tight_layout()
    plot_name = 'sv_led_fit_1_{}.png'.format(scale)
    plt.savefig(out_folder + plot_name)
    plt.rcParams['savefig.dpi'] = 100
    plt.show()
    plt.close()

## Plot response to spectral validation LEDs (individual)

In [ ]:
from scipy.interpolate import interp1d

plt.rcParams['savefig.dpi'] = 600
plt.rcParams['figure.figsize'] = (1.5, 1.5)
plt.rcParams['axes.color_cycle'] = sv_cycle
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

plt.rcParams['font.size'] = 12
plt.rcParams['xtick.labelsize'] = plt.rcParams['font.size']
plt.rcParams['ytick.labelsize'] = plt.rcParams['font.size']

df1 = dfs['sv-fas'].sort_values(['Spectral intensity','Top LED ID'])
df2 = dfs['sv-ras'].sort_values(['Spectral intensity','Top LED ID'])

for i in range(8):
    
    name = sv_dfs[i]['name'][0]
    
    imax = 13
    Xd1 = np.reshape(df1['Spectral intensity'], (12,8))[:imax,i]
    Xd2 = np.reshape(df2['Spectral intensity'], (12,8))[:imax,i]
    Yd1 = np.reshape(df1['FL1 Mean'], (12,8))[:imax,i]
    Yd2 = np.reshape(df2['FL1 Mean'], (12,8))[:imax,i]

    Xm = np.reshape(df1['Spectral intensity'], (12,8))[:,i]
    Xm = np.logspace(np.log10(Xm[0]), np.log10(Xm[-1]), 100)
    Ym1 = sims['sv-fas']
    Ym2 = sims['sv-ras']
    
    # Calculate/print error metrics
    Yi1 = interp1d(Xm,Ym1[i])
    Yi2 = interp1d(Xm,Ym2[i])
    
    errs1_abs = Yd1 - Yi1(Xd1)
    errs2_abs = Yd2 - Yi2(Xd2) 
    errs_abs = np.append(errs1_abs, errs2_abs)
    errs_abs = errs_abs[~np.isnan(errs_abs)]
    
    errs1_log = np.log10(Yd1/Yi1(Xd1))
    errs2_log = np.log10(Yd2/Yi2(Xd2))
    errs_log = np.append(errs1_log, errs2_log)
    errs_log = errs_log[~np.isnan(errs_log)]
    
    nrmse_abs = np.sqrt(np.mean(errs_abs**2)) / m.params['a'].value
    rmse_log = np.sqrt(np.mean(errs_log**2))
    
#     print errs_abs
#     print errs_log
    print [name, 100*nrmse_abs, rmse_log, 100*(10**rmse_log-1)]
    
    plt.loglog(Xm, Ym1[i], '-', color = sv_cycle[i])
    plt.loglog(Xm, Ym2[i], '-', color = sv_cycle[i])
    plt.loglog(Xd1, Yd1, '.', color = sv_cycle[i], marker='.', ms=8, markeredgecolor='k')
    plt.loglog(Xd2, Yd2, '.', color = sv_cycle[i], marker='^', ms=5, markeredgecolor='k')
    
    ax = plt.gca()
    
    ax.set_ylim(plt.ylim([3e1,1.5e4]))
    ax.set_yticks([1e2,1e3,1e4])
    ax.set_yticklabels([])

    ax.set_xlim([5e-3, 4e1])
    ax.set_xticks([1e-2, 1e-1, 1e0, 1e1])
    ax.set_xticklabels([])

    ax.minorticks_on()
    ax.tick_params(axis='x',which='both',top='off')
    ax.tick_params(axis='y',which='both',right='off')

#     ax = plt.gca()
#     ax.text(0.5,0.95,\
#             r'{}'.format(led_names[i]),\
#             horizontalalignment='center',\
#             verticalalignment='top',\
#             fontsize = plt.rcParams['font.size'],
#             transform=ax.transAxes)

    plt.tight_layout()
    plot_name = 'sv_led_fit_{}.png'.format(name)
    plt.savefig(out_folder + plot_name, pad_inches=0, bbox_inches='tight')
#     plt.rcParams['savefig.dpi'] = 100
    plt.show()
    plt.close()